<a href="https://colab.research.google.com/github/shiomasa1218/Colaboratory/blob/master/DFT321.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# drive mean root directory of  google drive
!ls ./gdrive/'My Drive'/'Kumamoto-Univ'/'Graduationwork'/'exefolder'/'test_folder_name'

carpet1_2.0_afterlinear   sponge-y_2.0_afterlinear
carpet2_2.0_afterlinear   stonetile1_2.0_afterlinear
carpet3_2.0_afterlinear   whiteitile1_2.0_afterlinear
sponge-g_2.0_afterlinear  woodtile1_2.0_afterlinear


In [0]:
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# check auth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
cd gdrive/'My Drive'/'Kumamoto-Univ'/'Graduationwork'/'exefolder'/

/content/gdrive/My Drive/Kumamoto-Univ/Graduationwork/exefolder


In [5]:
import numpy as np
import os
import scipy.fftpack
from pylab import *
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt 
from scipy.signal import butter, filtfilt 
import math
import csv


fs = 330.0 # サンプリング周波数 333Hz => 3m秒間隔でサンプリング 

#通常用FFT
def ffta(data):
    return np.fft.fft(data)

def butter_lowpass(cutoff, fs, order=5): 
    nyq = 0.5 * fs 
    normal_cutoff = cutoff/nyq 
    b, a = butter(order, normal_cutoff, btype='low', analog=False) 
    return b, a 

def butter_lowpass_filtfilt(data, cutoff, fs, order=5): 
    b, a = butter_lowpass(cutoff, fs, order=order) 
    y = filtfilt(b, a, data) 
    return y 
  
def smtdA(csvdata):

    N = len(csvdata) # FFTのサンプル数 
    Ax = np.fft.fft(csvdata) 
    fs = 330.0 # サンプリング周波数 333Hz => 3m秒間隔でサンプリング 
#     freqList = scipy.fftpack.fftfreq(N, d=1.0/ fs) # 周波数軸の値を計算 

    Ax_abs = np.abs(Ax) # 振幅スペクトル
    Ax_abs_amp = Ax_abs / N * 2 # 交流成分はデータ数で割って2倍
    Ax_abs_amp[0] = Ax_abs_amp[0] / 2 # 直流成分（今回は扱わないけど）は2倍不要
    
    theta = np.angle(Ax)
    
#     plot(Ax_abs_amp)
#     show()
    
#     iAx=np.fft.ifft(Ax)
#     plot(np.abs(iAx))
#     plot(x_csv)
#     axis([0,500,-500,500])
#     show()

    cutoff = 10
    condA = butter_lowpass_filtfilt(Ax_abs_amp, cutoff, fs) #ゼロ位相遅延フィルタをかけて平滑化
#     print(condA*np.cos(theta))
    condA_real = condA*np.cos(theta)
    condA_imag = condA*np.sin(theta)
    
#     plot(Ax_abs_amp)
#     plot(condAx)
#     axis([30,len(Ax_abs_amp)/4+30,0,10])
#     show()
    return condA,condA_real,condA_imag


# 訓練データのフォルダのパス
train_folder_name = "train_folder_name"

# # 検証データのフォルダのパス
# test_folder_name = "test_folder_name"


trainFolder = os.listdir(train_folder_name)
# testFolder = os.listdir(test_folder_name)

os.makedirs("train_dft321")
for i,d in enumerate(trainFolder):
   
    if d != ".DS_Store":
        dir_path = 'train_dft321/' + d +'_dft321'
        os.makedirs(dir_path,exist_ok=True)
        #フォルダ内のファイルのリストを取得
        files = os.listdir(train_folder_name + '/'+d)
        print(files)
        #ファイル毎にみる
        for f in files:
                            
            #.DS_Storeをどかす
            if f != ".DS_Store":
                #どのファイルを見ているか確認用
                print("load:"+f)

                datafile_path = train_folder_name + '/' + d+'/'+f

                #csvから読み込み
                data = np.loadtxt(datafile_path, delimiter=",")     #pbldata用

                #データ取り出し
                x_csv = data[1, :]
                y_csv = data[2, :]
                z_csv = data[3, :]

                #smoothing
                Ax, real_x, imag_x = smtdA(x_csv)
                Ay, real_y, imag_y = smtdA(y_csv)
                Az, real_z, imag_z = smtdA(z_csv)
                #DFT321式（３）
                A_s = sqrt(np.abs(Ax)**2+np.abs(Ay)**2+np.abs(Az)**2)
                
                #
                real_sum = real_x+real_y+real_z
                imag_sum = imag_x+imag_y+imag_z
               
                theta = np.angle(real_sum+1j*imag_sum)
                
                ifftA = np.fft.ifft(A_s * np.exp(1j * theta))
                realA = ifftA.real
   

                #ここでDFT321
                DFT321 = []
                
                DFT321.append(realA)
                
                path_w = 'train_dft321/' + d +"_dft321" +'/' + f
                path_w = path_w.replace(".csv" , "_dft321.csv")
                np.savetxt(path_w, DFT321, delimiter=",")
                upload_file_2 = drive.CreateFile()
                upload_file_2.SetContentFile(path_w)
                upload_file_2.Upload()

            else:
                print(".ds_storeを除去")
    else:
        print("ds_storeを抹殺")
        
print("transformed")


['tcarpet1_241.csv', 'tcarpet1_29.csv', 'tcarpet1_255.csv', 'tcarpet1_269.csv', 'tcarpet1_282.csv', 'tcarpet1_296.csv', 'tcarpet1_254.csv', 'tcarpet1_240.csv', 'tcarpet1_297.csv', 'tcarpet1_283.csv', 'tcarpet1_256.csv', 'tcarpet1_268.csv', 'tcarpet1_243.csv', 'tcarpet1_257.csv', 'tcarpet1_295.csv', 'tcarpet1_28.csv', 'tcarpet1_281.csv', 'tcarpet1_242.csv', 'tcarpet1_246.csv', 'tcarpet1_294.csv', 'tcarpet1_280.csv', 'tcarpet1_290.csv', 'tcarpet1_285.csv', 'tcarpet1_252.csv', 'tcarpet1_253.csv', 'tcarpet1_247.csv', 'tcarpet1_284.csv', 'tcarpet1_293.csv', 'tcarpet1_287.csv', 'tcarpet1_291.csv', 'tcarpet1_250.csv', 'tcarpet1_278.csv', 'tcarpet1_244.csv', 'tcarpet1_251.csv', 'tcarpet1_245.csv', 'tcarpet1_292.csv', 'tcarpet1_222.csv', 'tcarpet1_279.csv', 'tcarpet1_286.csv', 'tcarpet1_236.csv', 'tcarpet1_237.csv', 'tcarpet1_223.csv', 'tcarpet1_221.csv', 'tcarpet1_230.csv', 'tcarpet1_234.csv', 'tcarpet1_235.csv', 'tcarpet1_218.csv', 'tcarpet1_220.csv', 'tcarpet1_231.csv', 'tcarpet1_224.csv', '